In [ ]:
!pip install ensemble_boxes

In [1]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [2]:
# ensemble csv files
# submission_swin_l_multi
submission_files = ['../../sample_submission/submission_swin_l_multi.csv',
                    '../../sample_submission/submission_0401_ensemble.csv',] # submission lists
submission_df = [pd.read_csv(file) for file in submission_files]

In [3]:
image_ids = submission_df[1]['image_id'].tolist()

In [4]:
# ensemble 할 file의 image 정보를 불러오기 위한 json
annotation = '../../dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [6]:
prediction_strings = []
file_names = []
# ensemble 시 설정할 iou threshold 이 부분을 바꿔가며 대회 metric에 알맞게 적용해봐요!
iou_thr = 0.55

## soft_nms & wbf
weights = [2,1]
skip_box_thr = 0.001
sigma = 0.1


# 각 image id 별로 submission file에서 box좌표 추출
for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
#     각 submission file 별로 prediction box좌표 불러오기
    for df in submission_df:
        try:
            predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        except:
            print(image_id)
            print(df[df['image_id'] == image_id]['PredictionString'].tolist())
            exit()
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))

#     예측 box가 있다면 이를 ensemble 수행
    if len(boxes_list) == 2:
        # boxes, scores, labels = nms(boxes_list, scores_list, labels_list, iou_thr=iou_thr) # nms!
        # boxes, scores, labels = soft_nms(boxes_list, scores_list, labels_list, iou_thr=iou_thr, sigma=sigma, thresh=skip_box_thr) # soft_nms!
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr, conf_type = 'max') # , conf_type = 'max'
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    ### weight를 1로 맞추라고 warning 뜨는경우...! (앙상블 대상 csv 중 특정 row 가 empty prediction일 때...)
    # elif len(boxes_list)==1:
    #     boxes_list.append([])
    #     scores_list.append([])
    #     labels_list.append([])
    #     boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    #     for box, score, label in zip(boxes, scores, labels):
    #         prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
        
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

In [7]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('ensemble_swint_hardaug_to(4_3_재욱)_wbf_thr_055_max_weighted.csv')

submission.head()

,PredictionString,image_id
0,7 0.999920129776001 603.8963623046875 512.9420...,test/0000.jpg
1,5 0.9996607303619385 335.338134765625 243.6771...,test/0001.jpg
2,1 0.9972907900810242 97.1069564819336 294.4400...,test/0002.jpg
3,9 0.9998637437820435 143.36895751953125 259.61...,test/0003.jpg
4,1 0.9992254972457886 180.70480346679688 267.72...,test/0004.jpg
